In [47]:
import pandas as pd
import math


births = pd.read_csv("data/birth_data.csv")
births.columns = ["Month", "Birth Rate"]
births.index = births["Month"]
births.drop(columns=["Month"], inplace=True)


jobs = pd.read_csv("data/Employment_no_SA.csv")
jobs = jobs.drop(["Month.1"], axis=1)
jobs.index = jobs.Month
jobs = jobs.drop(["Month"], axis=1)
jobs.columns = ["Employment %"]

bus_rides = pd.read_csv("data/passengers_per_month.csv")
bus_rides.columns = ["Month", "Avg Bus Utilization"]
bus_rides.index = bus_rides["Month"]
bus_rides.drop(columns=["Month"], inplace=True)


population = pd.read_csv("data/population_data.csv")
air = pd.read_csv("data/air_quality_data.csv")
air["DateTime"] = pd.to_datetime(air["DateTime"])
air.set_index("DateTime", inplace=True)

air_month = pd.DataFrame(air.resample("BMS").mean()["AQI_Site"])
air_month.columns = ["AQI"]
air_month.index = pd.to_datetime(air_month.index)

Serieses = [births, jobs]
# Serieses = [births, jobs, bus_rides]
for s in Serieses:
    print(s.head(3))


            Birth Rate
Month                 
2006-01-01         390
2006-02-01         364
2006-03-01         417
            Employment %
Month                   
1978-02-01          61.1
1978-03-01          61.1
1978-04-01          61.1


In [120]:
series_names = ["Employment %", "Birth Rate"]

doo = pd.concat(Serieses, axis=1)
doo.dropna(inplace=True)
projection = pd.DataFrame(columns=doo.columns)
for s_name in series_names:
    doo[s_name + " delta"] = doo[s_name].diff()
doo.drop(doo.head(1).index, inplace=True)
doo.drop(doo.tail(1).index, inplace=True)

/home/metaten/.virtualenvs/govhack/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
periods = 120
pred = pd.date_range(doo.tail(1).index.values[0], periods=periods, freq='M')
e_end: float = 70 
e_start: float = 60
def lerp(start, end, frac):
    return start + ((end - start) * frac)

def lerp_range(start, end, steps):
    return [lerp(start, end, x/(steps -1)) for x in range(steps)]

d = pd.DataFrame({"Date": pred, "Employment": lerp_range(e_start,e_end, periods)})

In [114]:
d

,Date,Employment
0,2017-11-30,60.000000
1,2017-12-31,60.084034
2,2018-01-31,60.168067
3,2018-02-28,60.252101
4,2018-03-31,60.336134
...,...,...
115,2027-06-30,69.663866
116,2027-07-31,69.747899
117,2027-08-31,69.831933
118,2027-09-30,69.915966


In [119]:
last = doo.tail(1)
predicted_series = "Employment %"
series_names = ["Employment %", "Birth Rate"]

for row in d.iterrows():
    print(row)
    diff = last[predicted_series] - row[predicted_series]
    doo["LS"] = ((doo[predicted_series] - diff)/ doo[predicted_series].mean()) ** 2
    doo[doo["LS"] == doo["LS"].min()]
    
    
    
    
    # projection.append()

(0, Date          2017-11-30 00:00:00
Employment                     60
Name: 0, dtype: object)
(1, Date          2017-12-31 00:00:00
Employment                 60.084
Name: 1, dtype: object)
(2, Date          2018-01-31 00:00:00
Employment                60.1681
Name: 2, dtype: object)
(3, Date          2018-02-28 00:00:00
Employment                60.2521
Name: 3, dtype: object)
(4, Date          2018-03-31 00:00:00
Employment                60.3361
Name: 4, dtype: object)
(5, Date          2018-04-30 00:00:00
Employment                60.4202
Name: 5, dtype: object)
(6, Date          2018-05-31 00:00:00
Employment                60.5042
Name: 6, dtype: object)
(7, Date          2018-06-30 00:00:00
Employment                60.5882
Name: 7, dtype: object)
(8, Date          2018-07-31 00:00:00
Employment                60.6723
Name: 8, dtype: object)
(9, Date          2018-08-31 00:00:00
Employment                60.7563
Name: 9, dtype: object)
(10, Date          2018-09-30 00:00:00
E

In [101]:

doo["LS"] = ((doo["E delta"] - 61)/ doo["E delta"].mean()) ** 2 \
            + ((doo["B delta"] - 400)/doo["B delta"].mean()) ** 2 
doo[doo["LS"] == doo["LS"].min()]

,Birth Rate,Employment %,E delta,B delta,LS
2011-09-01,474.0,65.5,0.1,59.0,3.802516e+07


In [100]:
doo

,Birth Rate,Employment %,E delta,B delta,LS
1978-03-01,NaN,61.1,0.0,NaN,NaN
1978-04-01,NaN,61.1,0.0,NaN,NaN
1978-05-01,NaN,61.1,0.0,NaN,NaN
1978-06-01,NaN,61.0,-0.1,NaN,NaN
1978-07-01,NaN,61.0,0.0,NaN,NaN
...,...,...,...,...,...
2019-02-01,NaN,65.7,0.0,NaN,NaN
2019-03-01,NaN,65.8,0.1,NaN,NaN
2019-04-01,NaN,65.9,0.1,NaN,NaN
2019-05-01,NaN,66.0,0.1,NaN,NaN


In [ ]:
def dist(series, target):
    return ((series - target[series.nam])/series.mean()) ** 2
doo.apply(lambda :)

In [ ]:
from typing import List

pred = pd.DataFrame()

class Month:
    def __init__(self, y, z):
        y: float = y
        z: float = z
    
    # def distance(self, other: Month) -> float:
    #     


sep = Month(400, )



new_state = Month()